**Table of contents**<a id='toc0_'></a>    
- [Libraries](#toc1_)    
- [Loading text filings](#toc2_)    
- [Extracting the Management's Discussion and Analysis Section](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [138]:
import os
import time
os.chdir(os.environ.get('PROJECT_PATH'))
from secnlp.ml_logic import data as d
from secnlp.ml_logic import parsing as p
from secnlp import utils as u
from secnlp.params import *
import pandas as pd

# <a id='toc2_'></a>[Loading text filings](#toc0_)

In [3]:
df = u.read_data_from_bq(credentials = SERVICE_ACCOUNT, gcp_project = PROJECT, bq_dataset = DATASET_ID, table = FILINGS_10KQ_TABLE_ID)

In [6]:
df['date_filed'] = pd.to_datetime(df['date_filed'])

In [20]:
filing_sample_10k = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-K')].sample(1)
filing_sample_10k['raw_filing'] = filing_sample_10k['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


In [21]:
filing_sample_10q = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-Q')].sample(1)
filing_sample_10q['raw_filing'] = filing_sample_10q['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


# <a id='toc3_'></a>[Extracting the Management's Discussion and Analysis Section](#toc0_)

In [139]:
text = filing_sample_10k['raw_filing'].iloc[0]

In [140]:
print(p.parse_10k_10q_filing_item7(text))

Item 7. Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations</span></div><div style="margin-top:12pt;text-align:justify"><span style="color:#000000;font-family:'Times New Roman',sans-serif;font-size:10pt;font-style:italic;font-weight:400;line-height:120%">The following discussion and analysis of our financial condition and results of operations should be read in conjunction with other sections of this Annual Report on Form 10-K, including our consolidated financial statements and the related notes and other financial information included elsewhere in this Annual Report on Form 10-K.</span></div><div style="margin-top:12pt;text-align:justify"><span style="color:#000000;font-family:'Times New Roman',sans-serif;font-size:10pt;font-style:italic;font-weight:400;line-height:120%">In addition to historical consolidated financial information, the following discussion contains forward-looking statements that reflect our plans, estimates, and beliefs. For i

Write a function that will delete all of the original markup language tags (HTML, XBRL, XML):
- [ ] Remove ASCII-Encoded segments – All document segment <TYPE> tags of GRAPHIC, ZIP, EXCEL, JSON, and PDF are deleted from the file. ASCII-encoding is a means of converting binary-type files into standard ASCII characters to facilitate transfer across various hardware platforms. A relatively small graphic can create a substantial ASCII segment. Filings containing multiple graphics can be orders of magnitude larger than those containing only textual information.
- [ ] Remove <DIV>, <TR>, <TD>, and <FONT> tags – Although we require some HTML information for subsequent parsing, the files are so large (and processed as a single string) that, for processing efficiency, we initially simply strip out some of the formatting HTML.
- [ ] Remove all XML – all XML embedded documents are removed.
- [ ] Remove all XBRL – all characters between <XBRL …> … </XBRL> are deleted.
- [ ] Remove SEC Header/Footer – All characters from the beginning of the original file thru </SEC-HEADER> (or </IMS-HEADER> in some older documents) are deleted from the file. Note however that the header information is retained and included in the tagged items discussed in section 4.1. In addition, the footer “-----END PRIVACY-ENHANCED MESSAGE-----” appearing at the end of each document is deleted.
- [ ] Replace \&NBSP and \&#160 with a blank space.
- [ ] Replace \&AMP and \&#38 with “&”
- [ ] Remove all remaining extended character references (ISO-8859-1, see https://www.sec.gov/info/edgar/specifications/edgarfm-vol2-v59.pdf, section "5.2.2.6 Extended Character Sets within HTML Documents".
Tag Exhibits – At this point in the parsing process all exhibits are tagged as discussed in section 3.2.
- [ ] Remove Markup Tags – remove all remaining markup tags (i.e., <…>).
- [ ] Remove excess linefeeds.